Deep Learning
=============

Assignment 5
------------

The goal of this assignment is to train a skip-gram model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [ ]:
%matplotlib inline
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import collections
import math
import numpy as np
import os
import random
import sys
import tensorflow as tf
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE

In [ ]:
# http://stackoverflow.com/questions/29772158/make-ipython-notebook-print-in-real-time
oldsysstdout = sys.stdout
class flushfile():
    def __init__(self, f):
        self.f = f
    def __getattr__(self,name): 
        return object.__getattribute__(self.f, name)
    def write(self, x):
        self.f.write(x)
        self.f.flush()
    def flush(self):
        self.f.flush()
sys.stdout = flushfile(sys.stdout)

Download the data from the source website if necessary.

In [ ]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception('Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

Read the data into a string.

In [ ]:
def read_data(filename):
    f = zipfile.ZipFile(filename)
    for name in f.namelist():
        return tf.compat.as_str(f.read(name)).split()
    f.close()
  
words = read_data(filename)
print('Data size %d' % len(words))

print("Words[:5]=", words[:5])

Build the dictionary and replace rare words with UNK token.

In [ ]:
vocabulary_size = 50000

def build_dataset(words):
    """Number all the words, then convert the text into a sequence of word numbers. Also count occurrences"""
    total_count = collections.Counter(words)
    print("Total vocabulary: ", len(total_count))
    count = [['UNK', -1]]    
    count.extend(total_count.most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count = unk_count + 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
    return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])
del words  # Hint to reduce memory.

Function to generate a training batch for the skip-gram model.

In [ ]:
def generate_batch(batch_size, num_skips, skip_window, sequence, index):
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(sequence[index])
        index = (index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [ skip_window ]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(sequence[index])
        index = (index + 1) % len(data)
    return batch, labels, index

print('data:', [reverse_dictionary[di] for di in data[:8]])

for skip_window in [1, 2]:
    data_index = 0
    batch, labels, data_index = generate_batch(8, 2 * skip_window, skip_window, data, data_index)
    print('\nwith skip_window = %d:' % (skip_window))
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])

Train a skip-gram model.

In [ ]:
batch_size = 128
embedding_size = 128 # Dimension of the embedding vector.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64 # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default(), tf.device('/cpu:0'):

    # Input data.
    train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Variables.
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    weights = tf.Variable(tf.truncated_normal(
            [vocabulary_size, embedding_size], stddev=1.0 / math.sqrt(embedding_size)))
    biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Model.
    # Look up embeddings for inputs.
    embed = tf.nn.embedding_lookup(embeddings, train_dataset)
    # Compute the softmax loss, using a sample of the negative labels each time.
    sampled_softmax = tf.nn.sampled_softmax_loss(weights, biases, embed, train_labels, num_sampled, vocabulary_size)
    loss = tf.reduce_mean(sampled_softmax)

    # Optimizer.
    optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)

    # Normalize the embeddings:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    
    # Compute the similarity between validation examples and all embeddings using cosine distance:
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [ ]:
num_steps = 100001
skip_window = 1 # How many words to consider left and right.
num_skips = 2

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    average_loss = 0
    for step in range(num_steps):
        batch_data, batch_labels, data_index = generate_batch(batch_size, num_skips, skip_window, data, data_index)
        feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += l
        if step % 2000 == 0:
            if step > 0:
                average_loss = average_loss / 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step %d: %f' % (step, average_loss))
            average_loss = 0
        # note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8 # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k+1]
                log = 'Nearest to %s:' % valid_word
                for k in xrange(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log = '%s %s,' % (log, close_word)
                print(log)

    final_embeddings = normalized_embeddings.eval()

In [ ]:
num_points = 400

tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
two_d_embeddings = tsne.fit_transform(final_embeddings[1:num_points+1, :])

In [ ]:
def plot(embeddings, labels):
    assert embeddings.shape[0] >= len(labels), 'More labels than embeddings'
    pylab.figure(figsize=(15,15))  # in inches
    for i, label in enumerate(labels):
        x, y = embeddings[i,:]
        pylab.scatter(x, y)
        pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points', ha='right', va='bottom')
    pylab.show()

words = [reverse_dictionary[i] for i in range(1, num_points+1)]
plot(two_d_embeddings, words)

---

Problem
-------

An alternative to Word2Vec is called [CBOW](http://arxiv.org/abs/1301.3781) (Continuous Bag of Words). In the CBOW model, instead of predicting a context word from a word vector, you predict a word from the sum of all the word vectors in its context. Implement and evaluate a CBOW model trained on the text8 dataset.

---

In [ ]:
def generate_cbow_batch(batch_size, num_skips, skip_window, sequence, index):
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size, num_skips), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(sequence[index])
        index = (index + 1) % len(data)
    for i in range(batch_size):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [ skip_window ]
        labels[i] = buffer[skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i][j] = buffer[target]
        buffer.append(sequence[index])
        index = (index + 1) % len(data)
    return batch, labels, index

print('data:', [reverse_dictionary[di] for di in data[:12]])

for skip_window in [1, 2]:
    cbow_index = 0
    cbow_batch, cbow_labels, cbow_index = generate_cbow_batch(8, 2 * skip_window, skip_window, data, cbow_index)
    print('\nwith skip_window = %d:' % (skip_window))
    print('    batch:', [[reverse_dictionary[bi] for bi in skips] for skips in cbow_batch])
    print('    labels:', [reverse_dictionary[li] for li in cbow_labels.reshape(8)])

In [ ]:
cbow_batch_size = 128
cbow_num_skips = 2
cbow_embedding_size = 128 # Dimension of the embedding vector.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
cbow_valid_size = 16 # Random set of words to evaluate similarity on.
cbow_valid_window = 100 # Only pick dev samples in the head of the distribution.
cbow_valid_examples = np.array(random.sample(range(cbow_valid_window), cbow_valid_size))
cbow_num_sampled = 64 # Number of negative examples to sample.

cbow_graph = tf.Graph()

with cbow_graph.as_default(), tf.device('/cpu:0'):

    # Input data.
    cbow_train = tf.placeholder(tf.int32, shape=[cbow_batch_size, cbow_num_skips])
    cbow_labels = tf.placeholder(tf.int32, shape=[cbow_batch_size, 1])
    cbow_valid = tf.constant(cbow_valid_examples, dtype=tf.int32)

    # Variables.
    cbow_embeddings = tf.Variable(tf.random_uniform([vocabulary_size, cbow_embedding_size], -1.0, 1.0))
    print("Embeddings shape: ", cbow_embeddings.get_shape())
    
    cbow_weights = tf.Variable(tf.truncated_normal(
            [vocabulary_size, cbow_embedding_size], stddev=1.0 / math.sqrt(cbow_embedding_size)))
    print("Weights shape: ", cbow_weights.get_shape())
    
    cbow_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Model.
    # Look up embeddings for inputs.
    cbow_embed = tf.nn.embedding_lookup(cbow_embeddings, cbow_train)
    print("Embed shape: ", cbow_embed.get_shape())
    cbow_embed = tf.reduce_sum(cbow_embed, 1)
    print("Embed shape: ", cbow_embed.get_shape())
    # Compute the softmax loss, using a sample of the negative labels each time.
    cbow_sampled_softmax = tf.nn.sampled_softmax_loss(
        cbow_weights, cbow_biases, cbow_embed, cbow_labels, cbow_num_sampled, vocabulary_size)
    print("Softmax shape: ", cbow_sampled_softmax.get_shape())
    cbow_loss = tf.reduce_mean(cbow_sampled_softmax)
    print("Loss shape: ", cbow_loss.get_shape())

    # Optimizer.
    cbow_optimizer = tf.train.AdagradOptimizer(1.0).minimize(cbow_loss)

    # Normalize the embeddings:
    cbow_norm = tf.sqrt(tf.reduce_sum(tf.square(cbow_embeddings), 1, keep_dims=True))
    cbow_normalized = cbow_embeddings / cbow_norm
    
    # Compute the similarity between validation examples and all embeddings using cosine distance:
    cbow_valid_embeddings = tf.nn.embedding_lookup(cbow_normalized, cbow_valid)
    cbow_similarity = tf.matmul(cbow_valid_embeddings, tf.transpose(cbow_normalized))

In [ ]:
num_steps = 100001
skip_window = 1 # How many words to consider left and right.
num_skips = 2

with tf.Session(graph=cbow_graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    average_loss = 0
    for step in range(num_steps):
        batch_data, batch_labels, cbow_index = generate_cbow_batch(cbow_batch_size, num_skips, skip_window, data, cbow_index)
        feed_dict = {cbow_train : batch_data, cbow_labels : batch_labels}
        _, l = session.run([cbow_optimizer, cbow_loss], feed_dict=feed_dict)
        average_loss += l
        if step % 2000 == 0:
            if step > 0:
                average_loss = average_loss / 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step %d: %f' % (step, average_loss))
            average_loss = 0
        # note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = cbow_similarity.eval()
            for i in xrange(cbow_valid_size):
                valid_word = reverse_dictionary[cbow_valid_examples[i]]
                top_k = 8 # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k+1]
                log = 'Nearest to %s:' % valid_word
                for k in xrange(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log = '%s %s,' % (log, close_word)
                print(log)

    cbow_final_embeddings = cbow_normalized.eval()

In [ ]:
cbow_num_points = 400

cbow_tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
cbow_two_d_embeddings = cbow_tsne.fit_transform(cbow_final_embeddings[1:cbow_num_points+1, :])

In [ ]:
def plot(embeddings, labels):
    assert embeddings.shape[0] >= len(labels), 'More labels than embeddings'
    pylab.figure(figsize=(15,15))  # in inches
    for i, label in enumerate(labels):
        x, y = embeddings[i,:]
        pylab.scatter(x, y)
        pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points', ha='right', va='bottom')
    pylab.show()

words = [reverse_dictionary[i] for i in range(1, cbow_num_points+1)]
plot(cbow_two_d_embeddings, words)